In [1]:
# Importing libraries
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter
import scipy
import json
from matplotlib import cm as CM
from image import *
from model import CSRNet
import torch
from tqdm import tqdm
%matplotlib inline

In [17]:
# Function to create density maps for images
def gaussian_filter_density(gt):
    print(gt.shape)
    density = np.zeros(gt.shape, dtype = np.float32)
    gt_count = np.count_nonzero(gt)
    if gt_count == 0:
        return density
    pts = np.array(list(zip(np.nonzero(gt)[1], np.nonzero(gt)[0])))
    leafsize = 2048
    # build kdtree
    tree = scipy.spatial.KDTree(pts.copy(), leafsize=leafsize)
    # query kdtree
    distances, locations = tree.query(pts, k = 4)
    
    print('generate density....')
    for i, pt in enumerate(pts):
        pt2d = np.zeros(gt.shape, dtype=np.float32)
        pt2d[pt[1], pt[0]] = 1.
        if gt_count > 1:
            sigma = (distances[i][1] + distances[i][2] + distances[i][3])*.1
        else:
            sigma = np.average(np.array(gt.shape))/2./2. #case: 1 point
        density += scipy.ndimage.filters.gaussian_filter(pt2d, sigma, mode='constant')
    print('Done!')
    return density

In [11]:
# setting the root to the Shangai dataset you have downlaoded
# change the root path as per your location of dataset

root = './ShanghaiTech/'

In [12]:
part_A_train = os.path.join(root, 'part_A/train_data', 'images')
part_B_train = os.path.join(root, 'part_A/test_data', 'images')
part_A_test = os.path.join(root, 'part_B/train_data', 'images')
part_B_test = os.path.join(root, 'part_B/test_data', 'images')
path_sets = [part_A_train, part_A_test]

In [13]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)

In [ ]:
for img_path in img_paths:
    print(img_path)
    mat = io.loadmat(img_path.replace('.jpg', '.mat').replace('images', 'ground-truth').replace('IMG', 'GT_IMG'))
    img = plt.imread(img_path)
    k = np.zeros((img.shape[0], img.shape[1]))
    gt = mat['image_info'][0,0][0,0][0]
    for i in range(0, len(gt)):
        if int(gt[i][1]) < img.shape[0] and int(gt[i][0]) < img.shape[1]:
            k[int(gt[i][1]), int(gt[i][0])] = 1
    k = gaussian_filter_density(k)
    with h5py.File(img_path.replace('.jpg', '.h5').replace('images', 'ground-truth'), 'w') as hf:
        hf['density'] = k

./ShanghaiTech/part_A/train_data\images\IMG_1.jpg
(768, 1024)
generate density....
Done!
./ShanghaiTech/part_A/train_data\images\IMG_10.jpg
(683, 1024)
generate density....
Done!
./ShanghaiTech/part_A/train_data\images\IMG_100.jpg
(654, 1024)
generate density....
Done!
./ShanghaiTech/part_A/train_data\images\IMG_101.jpg
(768, 1024)
generate density....
Done!
./ShanghaiTech/part_A/train_data\images\IMG_102.jpg
(768, 1024)
generate density....
Done!
./ShanghaiTech/part_A/train_data\images\IMG_103.jpg
(400, 400)
generate density....
Done!
./ShanghaiTech/part_A/train_data\images\IMG_104.jpg
(405, 540)
generate density....
Done!
./ShanghaiTech/part_A/train_data\images\IMG_105.jpg
(632, 990)
generate density....
Done!
./ShanghaiTech/part_A/train_data\images\IMG_106.jpg
(675, 1024)
generate density....
Done!
./ShanghaiTech/part_A/train_data\images\IMG_107.jpg
(517, 800)
generate density....
Done!
./ShanghaiTech/part_A/train_data\images\IMG_108.jpg
(767, 1024)
generate density....
Done!
./Shan

In [ ]:
plt.imshow(Image.open(img_paths[0]))